In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from config import db_pass
import pymysql

### Extract CSVs into DataFrames

In [2]:
nyse_file = "Resources/companylist_nyse.csv"
nyse_df = pd.read_csv(nyse_file)
nyse_df.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
0,DDD,3D Systems Corporation,10.75,1000.0,NaN,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/ddd
1,MMM,3M Company,190.21,200.0,NaN,Health Care,Medical/Dental Instruments,https://www.nasdaq.com/symbol/mmm
2,WBAI,500.com Limited,11.97,300.0,2013.0,Consumer Services,Services-Misc. Amusement & Recreation,https://www.nasdaq.com/symbol/wbai
3,WUBA,58.com Inc.,71.69,400.0,2013.0,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/wuba
4,EGHT,8x8 Inc,23.70,500.0,NaN,Technology,EDP Services,https://www.nasdaq.com/symbol/eght


### Transform premise DataFrame

### Create database connection

In [3]:
connection_string = f"root:{db_pass}@localhost/stocks_db"
engine = create_engine(f'mysql+pymysql://{connection_string}')

In [4]:
# Confirm tables
engine.table_names()

['all_stocks_5yr',
 'companylist_nasdq',
 'companylist_nyse',
 'industry_analysis']

### Load DataFrames into database

In [ ]:
nyse_df.to_sql(name='companylist_nyse', con=engine, if_exists='append', index=False)


# Remove unnamed column from dataframe since it is not valid
nyse_cols = ["Symbol", "Name", "LastSale", "MarketCap", "IPOyear", "Sector", "industry", "Summary Quote"]
nyse_transformed= nyse_df[nyse_cols].copy()

# Rename the column headers
#nyse_transformed = nyse_transformed.rename(columns={"Summary Quote": "Summar_quote"})

# Create a filtered dataframe from specific columns
#premise_cols = ["License Serial Number", "Premises Name", "County ID Code"]
#premise_transformed= premise_df[premise_cols].copy()

# Rename the column headers
nyse_transformed = nyse_transformed.rename(columns={"Summary Quote": "SummaryQuote"})

# Clean the data by dropping duplicates and setting the index
#premise_transformed.drop_duplicates("id", inplace=True)
#premise_transformed.set_index("id", inplace=True)

#nyse_transformed.loc[, 'MarketCap'] = 10

nyse_transformed.head()

# connection_string = "root:db_pass@localhost/stocks_db"
# engine = create_engine(f'mysql://{connection_string}')

host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

conn = pymysql.connect(
    host=host,
    port=int(3306),
    user="root",
    passwd=db_pass,
    db="stocks_db",
    charset='utf8mb4')

#nyse_transformed.to_sql(name='companylist_nyse', con=conn,  if_exists='replace', index = False)

try:
    with conn.cursor() as cursor:
        # Create a new record
        sql= "INSERT INTO `companylist_nyse` (`Symbol`, `Name`, `LastSale`,  `IPOyear`, `Sector`, `industry`, `Summary_Quote`) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, ('DDD', '3D Systems Corporation', 10.75,  'n/a', 'Technology', 'Computer Software: Prepackaged Software', 'https://www.nasdaq.com/symbol/ddd'))
        conn.commit()
   
finally:
     conn.close()
